In [ ]:
import boto3

# Initialize EC2 client
ec2 = boto3.resource('ec2', region_name='ap-northeast-2')

# Define the instance creation
def create_ec2_instance():
    # UserData script to install Docker, login to ECR, pull image, and run container
    user_data_script = '''#!/bin/bash
    sudo apt-get update -y
    sudo apt-get install -y docker.io
    sudo systemctl start docker
    sudo systemctl enable docker
    sudo apt-get install -y awscli jq
    $(aws ecr get-login-password --region ap-northeast-2 | sudo docker login --username AWS --password-stdin 975050024946.dkr.ecr.ap-northeast-2.amazonaws.com)
    sudo docker pull 975050024946.dkr.ecr.ap-northeast-2.amazonaws.com/microprofile:latest
    sudo docker run -d -p 3000:3000 975050024946.dkr.ecr.ap-northeast-2.amazonaws.com/microprofile:latest
    '''
    instance = ec2.create_instances(
        InstanceType='t2.micro',        
        KeyName='ChrisEvans',        
        MinCount=1,
        MaxCount=1,
        SecurityGroupIds=['sg-0fa22de49c689a551'], 
        UserData=user_data_script,         
        TagSpecifications=[
            {
                'ResourceType': 'instance',
                'Tags': [
                    {'Key': 'Name', 'Value': 'micromernfrontend'}
                ]
            }
        ]
    )
    print(f"Instance created with ID: {instance[0].id}")

create_ec2_instance()
